In [15]:
import pandas as pd
pmid_dataset = pd.read_csv('/data/jx4237data/TKG/TKG_JCDL/Bridge2AI_2m/pmid_dataset.csv.gz')

In [89]:
paper_bioentity = pd.read_csv('/data/jx4237data/TKG/TKG_JCDL/Bridge2AI_2m/paper_bioentity_95_with_url_deduplicated.csv.gz', dtype=str)

In [17]:
paper_bioentity

,PMID,Mention,Type,prob,EntityId,url
0,5,myohemerythrin,gene,0.973336935043335,NCBIGene3048,https://bioregistry.io/NCBIGene:3048
1,5,hemerythrin,gene,0.9804603457450868,NCBIGene7248,https://bioregistry.io/NCBIGene:7248
2,55,yeast glyceraldehyde-3-phosphate dehydrogenase,gene,0.9855808615684508,NCBIGene26330,https://bioregistry.io/NCBIGene:26330
3,55,yeast,species,0.9947445392608644,NCBITaxon4932,https://bioregistry.io/NCBITaxon:4932
4,55,glyceraldehyde-3-phosphate,drug,0.963684380054474,meshD005986,https://bioregistry.io/mesh:D005986
...,...,...,...,...,...,...
10406111,37748650,SMC5/6 complex,gene,0.9954244494438172,NCBIGene79677,https://bioregistry.io/NCBIGene:79677
10406112,37748650,tBRCT,gene,0.9879411458969116,NCBIGene10716,https://bioregistry.io/NCBIGene:10716
10406113,37748650,phosphoserines,drug,0.9882856607437134,CHEBI15811,https://bioregistry.io/CHEBI:15811
10406114,37748650,tBRCTs,gene,0.9883208870887756,NCBIGene1788,https://bioregistry.io/NCBIGene:1788


In [5]:
import numpy as np
# read the .npz file
tkg_embeddings_all = np.load('/data/jx4237data/Graph-CoT/Pipeline/updated_data/tkg_embeddings_all.npz')

# access the data from the .npz file
embeddings = tkg_embeddings_all['embeddings']
ids = tkg_embeddings_all['ids']

tkg_embeddings_new = np.load('/data/jx4237data/TKG/TKG_JCDL/Bridge2AI_10k/tkg_updated_360k_embeddings_all.npz')
new_embeddings = tkg_embeddings_new['embeddings']
new_ids = tkg_embeddings_new['ids']
embeddings_dict = {id_: embedding for id_, embedding in zip(ids, embeddings)}
new_embeddings_dict = {str(id_): embedding for id_, embedding in zip(new_ids, new_embeddings)}
combined_embeddings_dict = {**embeddings_dict, **new_embeddings_dict}

In [18]:
import pandas as pd
import numpy as np

def cumulative_distribution_top_n(series, n):
    """
    Calculates the cumulative distribution of the top n most frequent values in a pandas Series.

    Args:
        series (pd.Series): The pandas Series containing the data.
        n (int): The number of top values to consider.

    Returns:
        pd.Series: A pandas Series representing the cumulative distribution of the top n values.
    """
    value_counts = series.value_counts()
    top_n_counts = value_counts.head(n)
    cumulative_sum = top_n_counts.cumsum()
    total_count = series.count()
    cumulative_percentage = (cumulative_sum / total_count) * 100
    return cumulative_percentage

top_50_cumulative = cumulative_distribution_top_n(paper_bioentity['EntityId'], 5000)

# Print the result
print(top_50_cumulative)

# Example to get the percentage of the top 50 entities.
if len(top_50_cumulative) > 0:
    print(f"The top {len(top_50_cumulative)} entities account for {top_50_cumulative.iloc[-1]:.2f}% of all entities.")
else:
    print("There are less than 50 unique entities.")

EntityId
NCBITaxon9606             9.722600
meshD009369              12.094551
NCBITaxon10095           12.902211
NCBITaxon10090           13.626669
NCBITaxon10116           14.091905
                           ...    
NCBIGene1647             79.751523
meshD009081_mim252650    79.754291
NCBIGene8692             79.757058
NCBIGene9733             79.759826
CHEBI39949               79.762593
Name: count, Length: 5000, dtype: float64
The top 5000 entities account for 79.76% of all entities.


In [19]:
top_entity_set = set(top_50_cumulative.keys())

In [20]:
paper_bioentity = paper_bioentity[paper_bioentity['EntityId'].isin(top_entity_set)]

In [21]:
entity_pmids = paper_bioentity.groupby('EntityId')['PMID'].apply(list).to_dict()

In [25]:
paper_bioentity.drop_duplicates(subset=['EntityId'], keep='first', inplace=True)

/tmp/ipykernel_76181/374449535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paper_bioentity.drop_duplicates(subset=['EntityId'], keep='first', inplace=True)


In [29]:
entity_id_rank = list(entity_pmids.keys())

In [ ]:
import pandas as pd
import numpy as np

def sort_dataframe_by_long_ordered_list(df, column_name, ordered_list):
    """
    Sorts a pandas DataFrame by a specified column according to a long ordered list.

    Args:
        df (pd.DataFrame): The DataFrame to sort.
        column_name (str): The name of the column to sort by.
        ordered_list (list): The desired order of values in the column (length 5000 in this case).

    Returns:
        pd.DataFrame: The sorted DataFrame.
    """

    # Create a mapping dictionary for efficient lookup
    order_map = {val: index for index, val in enumerate(ordered_list)}

    # Create a new column with the order index
    df['order_index'] = df[column_name].map(order_map)

    # Sort the DataFrame by the order index
    df_sorted = df.sort_values(by='order_index')

    # Drop the temporary order_index column
    df_sorted = df_sorted.drop(columns='order_index')

    return df_sorted


paper_bioentity = sort_dataframe_by_long_ordered_list(paper_bioentity, 'EntityId', entity_id_rank)

/tmp/ipykernel_76181/2565313040.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['order_index'] = df[column_name].map(order_map)


,PMID,Mention,Type,prob,EntityId,url
78,1140,choline,drug,0.9989407658576964,CHEBI133341,https://bioregistry.io/CHEBI:133341
31846,627060,Quinine,drug,0.999497652053833,CHEBI137041,https://bioregistry.io/CHEBI:137041
1275,16889,acetate,drug,0.9964773058891296,CHEBI15366,https://bioregistry.io/CHEBI:15366
9113,165356,PGE-2,drug,0.9995617270469666,CHEBI15551,https://bioregistry.io/CHEBI:15551
9115,165356,PGF-2-alpha,drug,0.9997817873954772,CHEBI15553,https://bioregistry.io/CHEBI:15553
...,...,...,...,...,...,...
4639,81356,type B interruption of the aortic arch,disease,0.99985009431839,mim217095,https://bioregistry.io/mim:217095
123387,8625125,MPMs,disease,0.9999969005584716,mim605711,https://bioregistry.io/mim:605711
177777,8795357,CRIB,disease,0.9993176460266112,mim609165,https://bioregistry.io/mim:609165
58297,8413957,mesial temporal lobe epilepsy,disease,0.9999971389770508,mim611630,https://bioregistry.io/mim:611630


In [23]:
# entity_pmids

In [31]:
import numpy as np
from tqdm import tqdm
entity_embeddings = {}
for entityId, pmids in tqdm(entity_pmids.items()):
    embeddings = [combined_embeddings_dict[pmid] for pmid in pmids]
    entity_embeddings[entityId] = np.sum(embeddings, axis=0)

100%|██████████| 5000/5000 [00:35<00:00, 141.22it/s]


In [32]:
index_to_entity = {}
current_index = 223000000000
for dataset in entity_embeddings.keys():
    index_to_entity[current_index] = dataset
    current_index += 1

In [49]:
entity_to_index = {dataset: index for index, dataset in index_to_entity.items()}

In [34]:
import pickle
pickle.dump(entity_embeddings, open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/entity_embeddings.pkl', 'wb'))

In [68]:
entity_embeddings = pickle.load(open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/entity_embeddings.pkl', 'rb'))

In [35]:
pickle.dump(index_to_entity, open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/index_to_entity.pkl', 'wb'))

In [7]:
import pickle

index_to_entity = pickle.load(open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/index_to_entity.pkl', 'rb'))

# metadata file

In [2]:
import pandas as pd
meta_data_df = pd.read_csv('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/work/data/author_dataset_filtered.csv')

In [4]:
meta_data_df.columns

Index(['id', 'FullName', 'BeginYear', 'PaperNum', 'CM4AI', 'RecentYear',
       'Index', 'is_author', 'index', 'color_category', 'Affiliation',
       'Data_Source', 'Data_Description', 'Data_url', 'pmids_string'],
      dtype='object')

In [32]:
# index_to_entity

In [39]:
paper_bioentity

,PMID,Mention,Type,prob,EntityId,url
78,1140,choline,drug,0.9989407658576964,CHEBI133341,https://bioregistry.io/CHEBI:133341
31846,627060,Quinine,drug,0.999497652053833,CHEBI137041,https://bioregistry.io/CHEBI:137041
1275,16889,acetate,drug,0.9964773058891296,CHEBI15366,https://bioregistry.io/CHEBI:15366
9113,165356,PGE-2,drug,0.9995617270469666,CHEBI15551,https://bioregistry.io/CHEBI:15551
9115,165356,PGF-2-alpha,drug,0.9997817873954772,CHEBI15553,https://bioregistry.io/CHEBI:15553
...,...,...,...,...,...,...
4639,81356,type B interruption of the aortic arch,disease,0.99985009431839,mim217095,https://bioregistry.io/mim:217095
123387,8625125,MPMs,disease,0.9999969005584716,mim605711,https://bioregistry.io/mim:605711
177777,8795357,CRIB,disease,0.9993176460266112,mim609165,https://bioregistry.io/mim:609165
58297,8413957,mesial temporal lobe epilepsy,disease,0.9999971389770508,mim611630,https://bioregistry.io/mim:611630


In [41]:
bio_data = {
    'id': list(index_to_entity.keys()),
    'FullName': paper_bioentity['Mention'].tolist(), # use the name
    'BeginYear': [2020] * len(index_to_entity),
    'PaperNum': [100] * len(index_to_entity),
    'CM4AI': [0] * len(index_to_entity),
    'RecentYear': [0] * len(index_to_entity),
    'Index': [0] * len(index_to_entity),
    'is_author': [0] * len(index_to_entity),
    'index': [0] * len(index_to_entity),
    'color_category': [6] * len(index_to_entity),
    'Affiliation': [''] * len(index_to_entity),
    'Data_Source': paper_bioentity['Type'].tolist(), 
    'Data_Description': list(index_to_entity.values()), # Entity ID.
    'Data_url': paper_bioentity['url'].tolist(),
    'pmids_string': [''] * len(index_to_entity),
}

df_bio_data = pd.DataFrame(bio_data)

# Append the new DataFrame to the original DataFrame
df_bio_data = pd.concat([meta_data_df, df_bio_data], ignore_index=True)

In [43]:
df_bio_data['Index'] =df_bio_data.index
df_bio_data['index'] =df_bio_data.index

In [82]:
df_bio_data.value_counts('color_category')

color_category
0    20924
5     7364
6     5000
1     1138
2       77
3       25
4        3
Name: count, dtype: int64

In [44]:
df_bio_data.to_csv('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/work/data/author_dataset_bio_entity.csv', index=False)

# add users and future users

In [90]:
import pandas as pd
paper_author_2m = pd.read_csv('/data/jx4237data/TKG/TKG_JCDL/Bridge2AI_2m/paper_author.csv.gz')


In [92]:
len(paper_bioentity)

10406116

In [91]:
paper_bioentity['entity_index'] = paper_bioentity['EntityId'].apply(lambda x: entity_to_index.get(x))

In [51]:
all_author_pd = pd.read_csv('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/work/data/author_dataset_bio_entity.csv')
aid_set = all_author_pd['id'].to_list()

In [95]:
len(paper_author_2m)

2933854

In [94]:
paper_author_2m = paper_author_2m[paper_author_2m['AID'].isin(aid_set)]

In [96]:
paper_bioentity['PMID'] = paper_bioentity['PMID'].astype(int)

In [97]:
merged_df = pd.merge(paper_author_2m, paper_bioentity, on='PMID')

# Group by dataset_id and collect unique AIDs
bio_id2users = merged_df.groupby('entity_index')['AID'].apply(lambda x: x.unique().tolist()).to_dict()

In [99]:
# len(bio_id2users)

In [63]:
import pandas as pd
import pickle
import gzip

author_collab = pickle.load(open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/author_dataset_collaborators.pkl', 'rb'))

In [80]:
# len(bio_id2users)

In [104]:
author_collab = { **author_collab, **bio_id2users }

In [105]:
pickle.dump(author_collab, open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/author_dataset_bioentity_collaborators.pkl', 'wb'))

In [106]:
authors = pd.read_csv('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/work/data/author_dataset_bio_entity.csv')
author_set = set(authors['id'].to_list())
def clean_author_collab(author_collab, author_set):
    # Remove keys not in author_set
    keys_to_remove = [key for key in author_collab if key not in author_set]
    for key in keys_to_remove:
        del author_collab[key]

    # Remove values not in author_set
    for key in author_collab:
        author_collab[key] = [value for value in author_collab[key] if value in author_set]

# Clean the dictionary
clean_author_collab(author_collab, author_set)

In [107]:
import json
file_name = '/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/work/data/author_collab_dataset_bioentity.json'

# Open the file in write mode and use json.dump to write the dictionary to the file
# write to /home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/work/data/author_collab_dataset_bioentity.json
with open(file_name, 'w') as json_file:
    json.dump(author_collab, json_file, indent=4)


In [108]:
dataset_index_embeddings = {}
for index, dataset in index_to_entity.items():
    dataset_index_embeddings[index] = entity_embeddings[dataset]

In [71]:
# dataset_index_embeddings

In [109]:
with gzip.open("/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/author_embeddings.pkl.gz", "rb") as f:
    author_ebd = pickle.load(f)
author_ebd_filtered = {key: value for key, value in author_ebd.items() if key in author_set}

import numpy as np

def get_top_n_similar_authors(author_ebd_filtered, dataset_id, N=5):
    # Check if the author_id exists in the dictionary
    if dataset_id not in dataset_index_embeddings:
        raise ValueError(f"Author ID {dataset_id} not found in the author embeddings.")
    
    # Get the embedding of the target author
    target_embedding = dataset_index_embeddings[dataset_id]
    
    # Initialize a list to store similarities
    similarities = []
    
    # Compute cosine similarity between the target author and all other authors
    for other_id, other_embedding in author_ebd_filtered.items():
        if other_id != dataset_id and  other_id not in author_collab[dataset_id]:  # Exclude the target author from the comparison
            cosine_similarity = np.dot(target_embedding, other_embedding) / (np.linalg.norm(target_embedding) * np.linalg.norm(other_embedding))
            similarities.append((other_id, cosine_similarity))
    
    # Sort the list by similarity in descending order and get the top N
    similarities.sort(key=lambda x: x[1], reverse=True)
    top_n_similar_authors = [author_id for author_id, _ in similarities[:N]]
    
    return top_n_similar_authors


In [110]:
import csv
import gzip
from tqdm import tqdm

import numpy as np

# Open a gzip-compressed file for writing
with gzip.open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/bioentity_author_recommendations.csv.gz', 'wt', newline='') as gzfile:
    writer = csv.writer(gzfile)
    
    # Write the header
    writer.writerow(['BioID', 'Recommended AID'])
    
    # Iterate over each author ID in author_collab
    for datasetid in tqdm(dataset_index_embeddings.keys()):
        # Get the top 30 recommended authors for the current author_id
        recommended_authors = get_top_n_similar_authors(author_ebd_filtered, datasetid, N=30)
        
        # Write each pair (author_id, recommended_id) to the CSV
        for recommended_id in recommended_authors:
            writer.writerow([datasetid, recommended_id])

100%|██████████| 5000/5000 [1:12:00<00:00,  1.16it/s]
